In [2]:
# Cài thư viện cần (dựa trên code trong repo)
!pip install transformers torch datasets accelerate

In [3]:
# Kiểm tra GPU có sẵn không
import torch
print("GPU available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU name:", torch.cuda.get_device_name(0))

# Cài lại các thư viện cần (nếu chưa chạy trước đó)
!pip install transformers torch datasets accelerate pandas scikit-learn tqdm

# Nếu bạn đã ở trong thư mục repo thì bỏ dòng này
%cd /content/Big-Five-personality-score-predictions_RoBERTa_and_BERT  # thay đổi nếu cần

GPU available: True
GPU name: Tesla P100-PCIE-16GB
[Errno 2] No such file or directory: '/content/Big-Five-personality-score-predictions_RoBERTa_and_BERT # thay đổi nếu cần'
/kaggle/working


In [4]:
# Đường dẫn đến file trong folder WevMining (thay tên file nếu khác)
file_path = "/kaggle/input/dataset/dataset (1).csv"  # hoặc .parquet nếu là Parquet

import pandas as pd

# Load file từ Drive
df = pd.read_csv(file_path)  # Nếu là Parquet: pd.read_parquet(file_path)

print("Cột gốc trong file:", df.columns.tolist())

# Rename cột để khớp với code/repo mong đợi (body cho text)
df = df.rename(columns={
    'text': 'body',
    'A': 'agr',
    'O': 'ope',
    'C': 'con',
    'E': 'ext',
    'N': 'neu'
})

df = df.sample(frac=0.1, random_state=42)

# Kiểm tra sau rename
print("\nCột sau khi rename:", df.columns.tolist())
print("\nSample 3 dòng đầu:")
print(df.head(3))


Cột gốc trong file: ['ope', 'con', 'ext', 'agr', 'neu', 'ptype', 'body', '__index_level_0__']

Cột sau khi rename: ['ope', 'con', 'ext', 'agr', 'neu', 'ptype', 'body', '__index_level_0__']

Sample 3 dòng đầu:
          ope   con   ext   agr   neu  ptype  \
258181    2.0  38.0  87.0  28.0   1.0      4   
768973   81.0  80.0  30.0  24.0  75.0     25   
1732080  29.0  40.0   6.0   7.0  98.0      1   

                                                      body  __index_level_0__  
258181   People who know they're close to zero balance ...            1386091  
768973                        Aww You are most welcome. =]            2044374  
1732080            Oh, didn't know it blocked her ult. TIL            1512060  


In [5]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer, RobertaModel
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from torch.optim import AdamW
from tqdm import tqdm
import os

# Load dataset đã rename
df[['ext', 'neu', 'agr', 'con', 'ope']] = df[['ext', 'neu', 'agr', 'con', 'ope']]/100

print("Dataset ready. Shape:", df.shape)

2026-01-19 07:58:46.615214: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768809526.956912      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768809527.056360      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768809527.918523      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768809527.918581      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768809527.918583      55 computation_placer.cc:177] computation placer alr

Dataset ready. Shape: (192420, 8)


In [6]:
# Drop author nếu còn (dataset HF thường không có)
if 'author' in df.columns:
    df.drop(['author'], axis=1, inplace=True)

df_train, df_test = train_test_split(df, test_size=0.3, random_state=42)
df_train, df_val = train_test_split(df_train, test_size=0.2, random_state=42)

print(f"Train: {len(df_train)} | Val: {len(df_val)} | Test: {len(df_test)}")

Train: 107755 | Val: 26939 | Test: 57726


In [7]:
# Cell: Train và Eval Baseline SVM (Linear) + TF-IDF
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import LinearSVR
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline

# Chuẩn bị data (giữ nguyên)
traits = ['ext', 'neu', 'agr', 'con', 'ope']
X_train = df_train['body'].astype(str)
y_train = df_train[traits].values
X_test = df_test['body'].astype(str)
y_test = df_test[traits].values

# Pipeline: Dùng LinearSVR với loss='squared_epsilon_insensitive' để fix lỗi dual=False
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=5000, ngram_range=(1, 2), stop_words='english')),
    ('svr', MultiOutputRegressor(LinearSVR(
        C=1.0,
        dual=False,
        loss='squared_epsilon_insensitive',  # <--- QUAN TRỌNG: Thêm dòng này để fix lỗi
        max_iter=10000
    )))
])

# Train baseline
print("Training LinearSVM + TF-IDF baseline...")
pipeline.fit(X_train, y_train)

# Predict
preds = pipeline.predict(X_test)

# Tính metrics
overall_rmse = np.sqrt(mean_squared_error(y_test, preds))
overall_r2 = r2_score(y_test, preds, multioutput='uniform_average')

print(f"Baseline LinearSVM+TF-IDF - Overall RMSE: {overall_rmse:.4f}")
print(f"Baseline LinearSVM+TF-IDF - Overall R²: {overall_r2:.4f}")

# Per-trait RMSE và R²
trait_names = ["Ext", "Neu", "Agr", "Con", "Ope"]
for i, name in enumerate(trait_names):
    trait_rmse = np.sqrt(mean_squared_error(y_test[:, i], preds[:, i]))
    trait_r2 = r2_score(y_test[:, i], preds[:, i])
    print(f"{name} - RMSE: {trait_rmse:.4f} | R²: {trait_r2:.4f}")

Training LinearSVM + TF-IDF baseline...
Baseline LinearSVM+TF-IDF - Overall RMSE: 0.2968
Baseline LinearSVM+TF-IDF - Overall R²: 0.0060
Ext - RMSE: 0.2983 | R²: -0.0083
Neu - RMSE: 0.3093 | R²: 0.0088
Agr - RMSE: 0.2973 | R²: 0.0221
Con - RMSE: 0.2652 | R²: 0.0041
Ope - RMSE: 0.3117 | R²: 0.0032


In [8]:
# Cell: Train và Eval Baseline Ridge Regression + TF-IDF
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import Ridge # <--- Thay LinearSVR bằng Ridge
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline

# Chuẩn bị data (giữ nguyên từ các bước trước)
# Đảm bảo df_train, df_test đã được load
traits = ['ext', 'neu', 'agr', 'con', 'ope']
X_train = df_train['body'].astype(str)
y_train = df_train[traits].values
X_test = df_test['body'].astype(str)
y_test = df_test[traits].values

# Pipeline: Dùng Ridge Regression
# Ridge là Linear Regression nhưng có thêm L2 regularization (giống Logistic)
# giúp tránh overfitting khi dùng với TF-IDF
pipeline_ridge = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=5000, ngram_range=(1, 2), stop_words='english')),
    ('ridge', MultiOutputRegressor(Ridge(alpha=1.0))) # alpha là hệ số phạt (regularization)
])

# Train
print("Training Ridge Regression + TF-IDF baseline...")
pipeline_ridge.fit(X_train, y_train)

# Predict
preds_ridge = pipeline_ridge.predict(X_test)

# Tính metrics
overall_rmse = np.sqrt(mean_squared_error(y_test, preds_ridge))
overall_r2 = r2_score(y_test, preds_ridge, multioutput='uniform_average')

print(f"Baseline Ridge+TF-IDF - Overall RMSE: {overall_rmse:.4f}")
print(f"Baseline Ridge+TF-IDF - Overall R²: {overall_r2:.4f}")

# Per-trait RMSE và R²
trait_names = ["Ext", "Neu", "Agr", "Con", "Ope"]
for i, name in enumerate(trait_names):
    trait_rmse = np.sqrt(mean_squared_error(y_test[:, i], preds_ridge[:, i]))
    trait_r2 = r2_score(y_test[:, i], preds_ridge[:, i])
    print(f"{name} - RMSE: {trait_rmse:.4f} | R²: {trait_r2:.4f}")

Training Ridge Regression + TF-IDF baseline...
Baseline Ridge+TF-IDF - Overall RMSE: 0.2962
Baseline Ridge+TF-IDF - Overall R²: 0.0096
Ext - RMSE: 0.2977 | R²: -0.0042
Neu - RMSE: 0.3087 | R²: 0.0121
Agr - RMSE: 0.2967 | R²: 0.0261
Con - RMSE: 0.2647 | R²: 0.0077
Ope - RMSE: 0.3111 | R²: 0.0066
